# Undirected and Directed Graphs
In this assignment we implement a number of graph data structures and algorithms, to represent and manipulate both undirected and directed graphs.


## Exercise – Union-Find Algorithm for Cycle Detection


We begin by representing a directed graph using adjacency lists. 

In [1]:
class Graph:
    def __init__(self, edges, N):
        self.adjList = [[] for _ in range(N)]
        # add edges to the graph
        for (u, v) in edges:
            self.adjList[u].append(v)


We then use the UF algorithm (lazy / quick union approach) as follow: first we put each vertex in its own disjoint set. We then iterate over all edges <tt>(u,v)</tt> in the graph, finding their roots; if <tt>root(u)==root(v)</tt>, then a cycle is found; otherwise we perform a union operation.

In [2]:
class UnionFind:
    def __init__(self, N):
        self.parent = [i for i in range(0, N)]

    # root operation
    def root(self, u):
        while u != self.parent[u]:
            u = self.parent[u]
        return u

    # find operation
    def find(self, u, v):
        return self.root(u)==self.root(v)

    # union operation
    def union(self, u,v):
        r_u = self.root(u)
        r_v = self.root(v)
        self.parent[r_u] = self.parent[r_v]

In [3]:
# returns true if graph has cycle
def findCycle(graph, N):

    # create singleton sets for each vertex in the graph
    uf = UnionFind(N)
 
    # consider every edge (u, v)
    for u in range(N):
        for v in graph.adjList[u]:
            # find roots of the sets to which elements u and v belong
            root_u = uf.root(u)
            root_v = uf.root(v)

            # if u and v have the same root, a cycle is found
            if root_u == root_v:
                return True
            else:
                uf.union(root_u, root_v)
    return False

In [5]:
# test code

# number of vertices in the graph
N = 12

edges = [(0, 1), (0, 6), (0, 7), (0, 2), (1, 5), (2, 3), (2, 4), (7, 8), (7, 11), (8, 9), (8, 10)]
graph = Graph(edges, N)

if findCycle(graph, N):
    print("Cycle found")
else:
    print("Cycle not found")
    
# edge (10, 11) introduces a cycle in the graph]
edges = [(0, 1), (0, 6), (0, 7), (0, 2), (1, 5), (2, 3), (2, 4), (7, 8), (7, 11), (8, 9), (8, 10), (10, 11)]
edges = [(0,1), (0,2), (1,2)]
graph = Graph(edges, N)

if findCycle(graph, N):
    print("Cycle found")
else:
    print("Cycle not found")



Cycle not found
Cycle found


## Exercise - is a graph G bipartite?
A bipartite graph is one where vertices can be divided into two disjoint subsets U and V, and all edges have one endpoint in U and the other endpoint in V.

To test whether G is bipartite, we begin by representing G using adjacency lists. 

In [ ]:
class Graph:
    def __init__(self, edges, N):
        self.adjList = [[] for _ in range(N)]
        # add edges to the graph
        for (u, v) in edges:
            self.adjList[u].append(v)
            self.adjList[v].append(u)

We then traverse G using BFS traversal (i.e., one level at a time). At each step, we "color" (label) the parent/child nodes with opposite colors (labels 0 and 1). If at any point we traverse an already colored (labelled) vertex with the same color, then the graph is not bipartite (same color basically represents being in the same set U or V). 

In [ ]:
from collections import deque

# BFS on graph starting from vertex v
def BFS(graph, v, N):

    # records whether vertices have been visited or not
    visited = [False] * N

    # recall color (0/1) of each vertex in BFS
    color = [None] * N

    # mark source vertex as visited and set its color to 0
    visited[v] = True
    color[v] = 0

    # deque-based implementation of BFS, performing level traversal of the graph
    q = deque()
    q.append(v)

    while len(q) > 0:

        v = q.popleft()

        # process each edge (v --> u) in turn
        for u in graph.adjList[v]:
            # if  u has not been visited yet
            if not visited[u]:
                visited[u] = True
                color[u] = (color[v] + 1) % 2
                q.append(u)

            # if u has already been visited and its color is the same of v, then the graph is not bipartite
            elif color[v] == color[u]:
                return False

    return True


In [ ]:
# test code
N = 10

# bipartite
edges = [(0,1), (0,4), (0,6), (2,9), (3,6), (3,1), (5,8), (7,4), (7,9)]
graph = Graph(edges, N)

if BFS(graph, 0, N):
    print("Graph is bipartite")
else:
    print("Graph is not bipartite")
    
# adding (3,0) makes G not bipartite
edges = [(0,1), (0,4), (0,6), (2,9), (3,6), (3,1), (3,0), (5,8), (7,4), (7,9)]
graph = Graph(edges, N)

if BFS(graph, 0, N):
    print("Graph is bipartite")
else:
    print("Graph is not bipartite")

## Exercise – Transitive Closure of a Graph

To find all vertices reachable from any given vertex <tt>u</tt>, we can use DFS (or BFS). By repeatedly calling DFS from every single vertex in G, we get the transitive closure of G. Time complexity: <tt>O(V(V+E))</tt> (which is much faster than <tt>V*V*V</tt> if G is sparse).

We first represent a directed graph as an adjacency list:

In [6]:
# adjacency list representation of an undirected graph
class Graph:
    def __init__(self, edges, N):
        self.adjList = [[] for _ in range(N)]

        for (u, v) in edges:
            self.adjList[u].append(v)

We then slightly modify DFS so to build the connectivity matrix CM as we explore the graph. For every child node reachable via DFS from the root, the node is added to CM.

In [7]:
def DFS(graph, CM, root, descendant):

    for child in graph.adjList[descendant]:
        if CM[root][child] == 0:
            CM[root][child] = 1
            DFS(graph, CM, root, child)

We then call the modified DFS starting from each vertex in turn.

In [8]:
# test code
N = 4
edges = [(0, 2), (1, 0), (3, 1)]
graph = Graph(edges, N)

# CM is the connectivity matrix and stores the transitive closure of the graph
# C[i][j] == 1 iif a directed path exists from vertex i to vertex j
CM = [[0 for u in range(N)] for v in range(N)]

# consider each vertex and start DFS from it
for v in range(N):
    CM[v][v] = 1
    DFS(graph, CM, v, v)

for v in range(N):
    print(CM[v])

[1, 0, 1, 0]
[1, 1, 1, 0]
[0, 0, 1, 0]
[1, 1, 1, 1]


In [9]:
# test code on a larger G
N = 12
edges = [(0, 1), (0, 6), (0, 7), (0, 2), (1, 5), (2, 3), (2, 4), (7, 8), (7, 11), (8, 9), (8, 10), (10, 11)]
graph = Graph(edges, N)

# CM is the connectivity matrix and stores the transitive closure of the graph
# C[i][j] == 1 iif a directed path exists from vertex i to vertex j
CM = [[0 for u in range(N)] for v in range(N)]

# consider each vertex and start DFS from it
for v in range(N):
    CM[v][v] = 1
    DFS(graph, CM, v, v)

for v in range(N):
    print(CM[v])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


## Exercise – IMDB Movie co-stars

We represent actors' co-starring relationships as an undirected graph, with vertices being actors, and an edge placed between two vertices iif the two actors have performed in the same movie. Since the graph will be very sparse, we adopt an adjacency list representation. To efficiently support the API, we will use a weighted undirected graph, with the weight being the number of movies two actors have co-starred. Finally, to compute the degrees of separation between any two actors, we will run BFS.

In [ ]:
class Graph:
    def __init__(self, edges):
        # we assume "egdges" is a list, with each element in the list being a tuple [(actor a, actor b), count]
        
        # we use a dict() so to use actor names as indices
        self.adjList = dict()

        for (a1, a2), count in edges.items():
            if  a1 in self.adjList:
                self.adjList[a1].append((a2, count)) 
            else:
                self.adjList[a1] = [(a2, count)]
                
            if a2 in self.adjList:
                self.adjList[a2].append((a1, count)) 
            else:
                self.adjList[a2] = [(a1, count)]
        
    def printGraph(self):
        for a, costars in self.adjList.items():
            print("actor", a, "played with:", costars)
            


In [ ]:
# realisation of API "Has actor a ever performed in a movie with actor b?"
def hasPerformed(G, a, b):
    if a in G.adjList:
        coactors = G.adjList[a]
        if coactors is None:
            return False
        for (actor, count) in coactors:
            if b == actor:
                return True
        return False
    return False


# realisation of API "In how many movies have a and b performed together?"
def moviesTogether(G, a, b):
    if a in G.adjList:
        coactors = G.adjList[a]
        if coactors is None:
            return 0
        for (actor, count) in coactors:
            if b == actor:
                return count
        return 0
    return 0

In [ ]:
# realisation of API "How many degrees of separation are there between actors a and b?"
# this relies on BFS

from collections import deque

def BFS(graph, v, N):
    # records whether vertices have been visited and at what distance from the source
    visited = dict()
    distance = dict()

    if v not in graph.adjList:
        return distance
    
    # mark source vertex as visited and set its distance to 0
    visited[v] = True
    distance[v] = 0

    # deque-based implementation of BFS, performing level traversal of the graph
    q = deque()
    q.append(v)

    while len(q)>0:

        v = q.popleft()

        # do for every edge v --> (u, c)
        for u, c in graph.adjList[v]:
            # if  u has not been visited yet
            if u not in visited:
                visited[u] = True
                distance[u] = distance[v] + 1
                q.append(u)
    return distance

def degreesApart(G, a, b):
    distance = BFS(G, a, len(G.adjList))
    if b in distance:
        return distance[b]
    else:
        return -1

In [ ]:
# pre-process 3-imdbtest.txt file once, so to put in format [actor a, actor b, number of movies together]
# the output file 4-imdbcostars.txt is then used in the test code

def preprocessimdb():
    with open('3-imdbtest.txt','r') as f:
        line = f.readline()
        edges = dict()
        while line != '':
            movie = line.split("/")
            actors = []
            # gather all actors who played in movie
            for i in range(1, len(movie)):
                actors.append(movie[i].rstrip("\n"))
            actors = sorted(actors)

            # create edges to re-present each co-starring
            for i in range(len(actors)):
                for j in range(i+1, len(actors)):
                    if (actors[i], actors[j]) not in edges:
                        edges[(actors[i], actors[j])]=1
                    else:
                        edges[(actors[i], actors[j])] = edges[(actors[i], actors[j])] + 1
            line = f.readline()

    with open('4-imdbcostars.txt','w') as f:
        for (a1, a2), count in edges.items():
            line = a1 + "/" + a2 + "/" + str(count) +"\n"
            f.write(line)

In [ ]:
#test code

preprocessimdb()

with open('4-imdbcostars.txt','r') as f:
    line = f.readline()
    edges =dict()
    while line != '':
        triplet = line.split("/")
        a1 = triplet[0]
        a2 = triplet[1]
        count = triplet[2]
        edges[(a1,a2)] = int(count)
        line = f.readline()
             
imdbGraph = Graph(edges)

print("pre-processing complete")


In [ ]:
# expected output: True / 1
print(hasPerformed(imdbGraph, "Gray, Ian (I)", "Haywood, Chris (I)"))
print(moviesTogether(imdbGraph, "Gray, Ian (I)", "Haywood, Chris (I)"))

# expected output: False / 0
print(hasPerformed(imdbGraph, "Gray, Ian (I)", "licia"))
print(moviesTogether(imdbGraph, "licia", "Gray, Ian (I)"))


In [ ]:
# expected output: 1 / 2 / 3
print(degreesApart(imdbGraph, "Gray, Ian (I)", "Haywood, Chris (I)"))
print(degreesApart(imdbGraph, "Gray, Ian (I)", "Venora, Diane"))
print(degreesApart(imdbGraph, "Free, Christopher", "Gray, Ian (I)"))